In [6]:
import pandas as pd
import pprint as pp

In [7]:
data = pd.read_csv("crunchbase-investments.csv", chunksize=500,encoding='ISO-8859-1')
print(type(data))
total_col_memory = dict()

for i in data:
    for col in i.columns:
        if col not in total_col_memory.keys():
            total_col_memory[col] = i[col].memory_usage(deep=True)/(1024**2)
        else:
            total_col_memory[col] += i[col].memory_usage(deep=True)/(1024**2)
total_memory = sum([v for v in total_col_memory.values()])

pp.pprint(total_col_memory)
print(" ")
print("total memory usage ", total_memory)

<class 'pandas.io.parsers.TextFileReader'>
{'company_category_code': 3.271106719970703,
 'company_city': 3.351999282836914,
 'company_country_code': 3.033710479736328,
 'company_name': 3.433443069458008,
 'company_permalink': 3.8782958984375,
 'company_region': 3.262028694152832,
 'company_state_code': 2.970648765563965,
 'funded_at': 3.3865785598754883,
 'funded_month': 3.2353248596191406,
 'funded_quarter': 3.2353248596191406,
 'funded_year': 0.4118537902832031,
 'funding_round_type': 3.2611913681030273,
 'investor_category_code': 0.5677452087402344,
 'investor_city': 2.7370290756225586,
 'investor_country_code': 2.51025390625,
 'investor_name': 3.742757797241211,
 'investor_permalink': 4.758308410644531,
 'investor_region': 3.247433662414551,
 'investor_state_code': 2.347476005554199,
 'raised_amount_usd': 0.4118537902832031}
 
total memory usage  57.05436420440674


In [22]:
data = pd.read_csv("crunchbase-investments.csv", chunksize=5000,encoding='ISO-8859-1')


cleaned_data = list()
for i in data:
    i.drop(["company_name","company_country_code","company_permalink","investor_permalink","funded_month","funded_quarter","funded_year"],inplace=True,axis=1)
    i["funded_at"] = pd.to_datetime(i["funded_at"])
    
    to_cat = ["company_category_code", "company_state_code","company_region","company_city",
              "investor_category_code","investor_country_code","investor_state_code","investor_region",
              "investor_city","funding_round_type","investor_name"]
    for j in to_cat:
        i[j] = i[j].astype("category").cat.codes
    cleaned_data.append(i)

    
pd.concat(cleaned_data).head()  

,company_category_code,company_state_code,company_region,company_city,investor_name,investor_category_code,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,raised_amount_usd
0,0,3,162,413,1,-1,43,28,138,221,5,2012-10-30,2000000.0
1,26,3,162,303,4,8,43,29,46,77,2,2012-01-23,20000.0
2,21,-1,195,-1,4,8,43,29,46,77,2,2012-01-01,20000.0
3,37,33,44,535,4,8,43,29,46,77,0,2012-02-15,20000.0
4,41,33,41,108,4,8,43,29,46,77,2,2011-09-08,20000.0


In [11]:
import sqlite3
import pprint as pp
conn = sqlite3.connect("crunchbase.db")
conn.rollback()
cur = conn.cursor()
chunk_iter = pd.read_csv("crunchbase-investments.csv", chunksize=5000,encoding='ISO-8859-1')

for chunk in chunk_iter:
    chunk.to_sql("investment",conn, if_exists="append", index=False)


In [18]:
cur.execute("SELECT * FROM investment LIMIT 1")
print(cur.fetchall(),"\n")
cur.execute("PRAGMA table_info(investment)")
pp.pprint(cur.fetchall())

[('/company/advercar', 'AdverCar', 'advertising', 'USA', 'CA', 'SF Bay', 'San Francisco', '/company/1-800-flowers-com', '1-800-FLOWERS.COM', None, 'USA', 'NY', 'New York', 'New York', 'series-a', '2012-10-30', '2012-10', '2012-Q4', 2012, 2000000.0)] 

[(0, 'company_permalink', 'TEXT', 0, None, 0),
 (1, 'company_name', 'TEXT', 0, None, 0),
 (2, 'company_category_code', 'TEXT', 0, None, 0),
 (3, 'company_country_code', 'TEXT', 0, None, 0),
 (4, 'company_state_code', 'TEXT', 0, None, 0),
 (5, 'company_region', 'TEXT', 0, None, 0),
 (6, 'company_city', 'TEXT', 0, None, 0),
 (7, 'investor_permalink', 'TEXT', 0, None, 0),
 (8, 'investor_name', 'TEXT', 0, None, 0),
 (9, 'investor_category_code', 'TEXT', 0, None, 0),
 (10, 'investor_country_code', 'TEXT', 0, None, 0),
 (11, 'investor_state_code', 'TEXT', 0, None, 0),
 (12, 'investor_region', 'TEXT', 0, None, 0),
 (13, 'investor_city', 'TEXT', 0, None, 0),
 (14, 'funding_round_type', 'TEXT', 0, None, 0),
 (15, 'funded_at', 'TEXT', 0, None, 0),
